In [5]:
import os
import csv
import json
import time
import math
import shutil
import numpy as np
import pandas as pd
from pathlib import Path

In [6]:
# ---------- go_no_go

def inter_stim_interval(num_rows):
    """
    Generate list of random inter-stim intervals between 4 and 6 seconds
    """

    inter_stim_times = []
    for i in range(num_rows):
        inter_stim_time = np.random.choice(np.arange(4, 7, 0.5))
        inter_stim_times.append(inter_stim_time)

    return inter_stim_times

def go_stims(num_rows, go_stim_dir):
    """
    Randomize the list of GO images
    """

    go_stim_list = os.listdir(go_stim_dir)
    np.random.shuffle(go_stim_list)

    return go_stim_list[:num_rows]

def go_stim_csv(num_rows, save_dir, go_stim_dir):
    """
    Save Go stim vectors in a CSV file
    """

    inter_stim_times = inter_stim_interval(num_rows)
    go_stim_list = go_stims(num_rows, go_stim_dir)

    filename = "go_stims-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["inter_stim_interval", "GNG_stim", "match", "corr_key"])  
        
        for inter_stim_time, go_stim in zip(inter_stim_times, go_stim_list):
            go_stim_str = os.path.join("go_stimuli", go_stim)
            csv_writer.writerow([inter_stim_time, go_stim_str, 1, "space"])  

    return filepath

def GNG_stims(num_rows, go_stim_dir, no_go_stim_dir):
    """
    Create a randomized list of 50/50 Go/No-Go images
    """

    # Go stims
    go_stim_list = os.listdir(go_stim_dir)
    np.random.shuffle(go_stim_list)
    go_stim_list = go_stim_list[:math.floor(num_rows/2)]

    # No-Go stims
    no_go_stim_list = os.listdir(no_go_stim_dir)
    np.random.shuffle(no_go_stim_list)
    no_go_stim_list = no_go_stim_list[:math.ceil(num_rows/2)]
    
    GNG_stim_list = go_stim_list + no_go_stim_list
    np.random.shuffle(GNG_stim_list)

    return GNG_stim_list

def GNG_stim_csv(num_rows, save_dir, go_stim_dir, no_go_stim_dir):
    """
    Save Go/No-Go stim vectors in a CSV file
    """

    inter_stim_times = inter_stim_interval(num_rows)
    GNG_stim_list = GNG_stims(num_rows, go_stim_dir, no_go_stim_dir)

    filename = "GNG_stims-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["inter_stim_inverval", "GNG_stim", "match", "corr_key"])  
        
        for inter_stim_time, GNG_stim in zip(inter_stim_times, GNG_stim_list):
            if "puppy" in GNG_stim:
                GNG_stim_str = os.path.join("go_stimuli", GNG_stim)
                csv_writer.writerow([inter_stim_time, GNG_stim_str, 1, "space"]) 
            elif "spider" in GNG_stim:
                GNG_stim_str = os.path.join("no_go_stimuli", GNG_stim)
                csv_writer.writerow([inter_stim_time, GNG_stim_str, 0, "None"]) 

    return filepath

def GNG_task_order_csv(cond_dir, save_dir):
    """
    Save TOL task order to a CSV file
    """

    cond_file_list = os.listdir(cond_dir)

    filename = "GNG_TO-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["task_order"])  
        
        half_length = int(len(cond_file_list)/2)
        for i in range(half_length):
            csv_writer.writerow([cond_file_list[i+half_length]])
            csv_writer.writerow([cond_file_list[i]])

    return filepath

# ---------- king_devick

def king_devick_task_order_csv(save_dir):
    """
    Save king devick card order in a CSV file
    """

    card_order = ["card_0.jpg", "card_1.jpg", "card_2.jpg", "card_3.jpg"]

    filename = "king_devick_TO.csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["task_order"]) 

        for card in card_order:
            csv_writer.writerow([card]) 

def king_devick_resp(par, save_dir):
    """
    Create a text file to log particpant response accuracy
    """

    file_lines = ["card", "------", "number incorrect: "]
    filename = par + "_king_devick_data.txt"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as file:
        for i in range(4):
            file.write(f"{file_lines[0]} {i}" + "\n")
            file.write(file_lines[1] + "\n")
            if i == 2:
                file.write(file_lines[2])
            else:
                file.write(file_lines[2] + "\n")
                file.write("\n")

# ---------- n_back

def zero_back(num_rows, match_prop=0.4):
    """
    Create zero-back vectors for number stimulus and match condition
    """

    match_num = np.random.randint(0, 10)

    can_pass = False
    while not can_pass:
        num_match_list = []
        total_match_count = 0
        for i in range(num_rows):
            num_match_temp = []
            num_stim = np.random.randint(0, 10)
            num_match_temp.append(num_stim)

            if num_stim == match_num:
                num_match_temp.append(1)
                total_match_count += 1
            else:
                num_match_temp.append(0)
            num_match_list.append(num_match_temp)
        
        # if match proportion criteria met
        if (total_match_count/num_rows) >= match_prop:  
            can_pass = True

    return match_num, num_match_list

def zero_back_csv(save_dir, num_rows, match_prop=0.4):
    """
    Save zero-back vectors in a CSV file
    """

    match_num, num_match_list = zero_back(num_rows, match_prop)
    filename = "ZB-" + "match_" + f"{str(match_num)}-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["num_stim", "match_num", "match", "corr_key"])  
        
        for num, match in num_match_list:
            if match == 0:
                csv_writer.writerow([num, match_num, match, "left"])  
            elif match == 1:
                csv_writer.writerow([num, match_num, match, "right"])

    return filepath

def one_back(num_rows, match_prop):
    """
    Create one-back vectors for number stimulus and match condition
    """

    can_pass = False
    while not can_pass:
        num_match_list = []
        total_match_count = 0
        for i in range(num_rows):
            num_match_temp = []
            num_stim = np.random.randint(0, 10)
            num_match_temp.append(num_stim)
            
            if i == 0:
                num_match_temp.append(0)
            else:
                if num_stim == num_match_list[i-1][0]:
                    num_match_temp.append(1)
                    total_match_count += 1
                else:
                    num_match_temp.append(0)
            num_match_list.append(num_match_temp)

        # if match proportion criteria met
        if (total_match_count/num_rows) >= match_prop:  
            can_pass = True

    return num_match_list

def one_back_csv(save_dir, num_rows, match_prop=0.4):
    """
    Save one-back vectors in a CSV file
    """

    num_match_list = one_back(num_rows, match_prop)
    filename = "OB-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["num_stim", "match", "corr_key"])  
        
        for num, match in num_match_list:
            if match == 0:
                csv_writer.writerow([num, match, "left"])  
            elif match == 1:
                csv_writer.writerow([num, match, "right"]) 

    return filepath

def two_back(num_rows, match_prop):
    """
    Create two-back vectors for number stimulus and match condition
    """

    can_pass = False
    while not can_pass:
        num_match_list = []
        total_match_count = 0
        for i in range(num_rows):
            num_match_temp = []
            num_stim = np.random.randint(0, 10)
            num_match_temp.append(num_stim)
            
            if i == 0 or i == 1:
                num_match_temp.append(0)
            else:
                if num_stim == num_match_list[i-2][0]:
                    num_match_temp.append(1)
                    total_match_count += 1
                else:
                    num_match_temp.append(0)
            num_match_list.append(num_match_temp)
        
        # if match proportion criteria met
        if (total_match_count/num_rows) >= match_prop:  
            can_pass = True

    return num_match_list

def two_back_csv(save_dir, num_rows, match_prop=0.4):
    """
    Save two-back vectors in a CSV file
    """

    num_match_list = two_back(num_rows, match_prop)
    filename = "TB-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["num_stim", "match", "corr_key"])  
        
        for num, match in num_match_list:
            if match == 0:
                csv_writer.writerow([num, match, "left"])  
            elif match == 1:
                csv_writer.writerow([num, match, "right"])

    return filepath

def n_back_task_order(cond_dir):
    """
    Random order of N-back tasks (task cannot repeat sequentially 3x)
    """

    cond_file_list = os.listdir(cond_dir)
    #task_list = ["zero_back1", "one_back1", "two_back1", "zero_back2", "one_back2", "two_back2", "zero_back3", "one_back3", "two_back3"]

    list_correct = False

    while not list_correct:
        np.random.shuffle(cond_file_list)

        for i, cond in enumerate(cond_file_list):
            if i == 0 or i == 1: 
                pass
            else:
                if "ZB" in cond and "ZB" in cond_file_list[i-1] and "ZB" in cond_file_list[i-2]:
                    break
                elif "OB" in cond and "OB" in cond_file_list[i-1] and "OB" in cond_file_list[i-2]:
                    break
                elif "TB" in cond and "TB" in cond_file_list[i-1] and "TB" in cond_file_list[i-2]:
                    break
                else:
                    pass
            if i == len(cond_file_list)-1:
                list_correct = True
                break

    return cond_file_list

def n_back_task_order_csv(cond_dir, save_dir):
    """
    Save vSAT stim vectors in a CSV file
    """

    cond_file_list = n_back_task_order(cond_dir=cond_dir)

    filename = "NB_TO-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["task_order"])  
        
        for cond in cond_file_list:
            csv_writer.writerow([cond])  

    return filepath 

# ---------- resting state

def resting_state_task_order_csv(save_dir):
    """
    Save resting state order in a CSV file
    """

    rest_conds = ["eyes open", "eyes closed"]
    np.random.shuffle(rest_conds)

    filename = "RS_TO-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["task_order"]) 

        for rest_cond in rest_conds:
            csv_writer.writerow([rest_cond]) 

    return filepath

# ---------- tower_of_london

def ZM_stims(num_rows, zero_move_dir):
    """
    Randomize the list of zero-move images
    """

    zero_move_list = os.listdir(zero_move_dir)
    np.random.shuffle(zero_move_list)

    return zero_move_list[:num_rows]

def ZM_stim_csv(num_rows, save_dir, zero_move_dir):
    """
    Save zero-move image vectors in a CSV file
    """

    zero_move_list = ZM_stims(num_rows, zero_move_dir)

    filename = "ZM_stims-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["image_stim", "match", "corr_key"])  
        
        for ZM_image in zero_move_list:
            zero_move_image_str = os.path.join("zero_move_stimuli", ZM_image)
            csv_writer.writerow([zero_move_image_str, 0, "left"]) 

    return filepath 

def MM_stims(num_rows, two_move_dir, not_two_move_dir):
    """
    Create a randomized list of 50/50 multi-move (two move and not two move) images
    """

    # two move
    two_move_image_list = os.listdir(two_move_dir)
    np.random.shuffle(two_move_image_list)
    two_move_image_list = two_move_image_list[:math.floor(num_rows/2)]

    # not two move
    not_two_move_image_list = os.listdir(not_two_move_dir)
    np.random.shuffle(not_two_move_image_list)
    not_two_move_image_list = not_two_move_image_list[:math.ceil(num_rows/2)]
    
    multi_move_image_list = two_move_image_list + not_two_move_image_list
    np.random.shuffle(multi_move_image_list)

    return multi_move_image_list

def MM_stim_csv(num_rows, save_dir, two_move_dir, not_two_move_dir):
    """
    Save multi-move image  vectors in a CSV file
    """

    multi_move_image_list = MM_stims(num_rows, two_move_dir, not_two_move_dir)

    filename = "MM_stims-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["image_stim", "match", "corr_key"])  
        
        for multi_move_image in multi_move_image_list:
            if "not" in multi_move_image:
                multi_move_image_str = os.path.join("multi_move_stimuli", "stim_not_two_move", multi_move_image)
                csv_writer.writerow([multi_move_image_str, 0, "left"]) 
            else:
                multi_move_image_str = os.path.join("multi_move_stimuli", "stim_two_move", multi_move_image)
                csv_writer.writerow([multi_move_image_str, 1, "right"]) 

    return filepath

def TOL_task_order_csv(cond_dir, save_dir):
    """
    Save TOL task order to a CSV file
    """

    cond_file_list = os.listdir(cond_dir)

    filename = "TOL_TO-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["task_order"])  
        
        half_length = int(len(cond_file_list)/2)
        for i in range(half_length):
            csv_writer.writerow([cond_file_list[i]])
            csv_writer.writerow([cond_file_list[i+half_length]])

    return filepath

# ---------- vSAT

def vSAT_stims(num_rows, SAT=False):
    """
    Random lists of vSAT conditions
    """

    stim_times = [0.017, 0.029, 0.050]
    inter_stim_times = [1, 2, 3]
    if SAT:
        stim_positions = [(0, 0)]  # middle only
    else:
        stim_positions = [(-0.25, 0.25), (0.25, 0.25), (0, 0), (-0.25, -0.25), (0.25, -0.25)]  # [top left, top right, middle, bottom left, bottom right]
    matches = [0, 1]

    stim_conditions_list = []
    inter_stim_conditions_list = []
    x_pos_list = []
    y_pos_list = []
    match_list = []

    for i in range(num_rows):
        stim_time = np.random.choice(stim_times)
        stim_conditions_list.append(stim_time)

        inter_stim_time = np.random.choice(inter_stim_times)
        inter_stim_conditions_list.append(inter_stim_time)

        np.random.shuffle(stim_positions)
        x_pos = stim_positions[0][0]
        x_pos_list.append(x_pos)
        y_pos = stim_positions[0][1]
        y_pos_list.append(y_pos)

        match = np.random.choice(matches)
        match_list.append(match)

    return stim_conditions_list, inter_stim_conditions_list, x_pos_list, y_pos_list, match_list

def vSAT_stim_csv(num_rows, save_dir, SAT=False):
    """
    Save vSAT stim vectors in a CSV file
    """

    stim_conditions_list, inter_stim_conditions_list, x_pos_list, y_pos_list, match_list = vSAT_stims(num_rows, SAT)

    if SAT:
        filename = "SAT_stims-" + str(time.time()).replace(".", "") + ".csv"
    else:
        filename = "vSAT_stims-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["stim_time", "inter_stim_time", "x_pos", "y_pos",  "match", "corr_key"])  
        
        for stim_time, inter_stim_time, x_pos, y_pos, match in zip(stim_conditions_list, inter_stim_conditions_list, x_pos_list, y_pos_list, match_list):
            if match == 1:
                csv_writer.writerow([stim_time, inter_stim_time, x_pos, y_pos, match, "right"])
            elif match == 0:   
                csv_writer.writerow([stim_time, inter_stim_time, x_pos, y_pos, match, "left"])

    return filepath

def vSAT_task_order_csv(cond_dir, save_dir):
    """
    Save vSAT stim vectors in a CSV file
    """

    cond_file_list = os.listdir(cond_dir)
    np.random.shuffle(cond_file_list)

    filename = "vSAT_TO-" + str(time.time()).replace(".", "") + ".csv"
    filepath = os.path.join(save_dir, filename)

    with open(filepath, mode="w", newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')  
        csv_writer.writerow(["task_order"])  
        
        for cond_file in cond_file_list:
            csv_writer.writerow([cond_file])   

    return filepath

In [7]:
def create_exp_dir(git_dir, par_name, exp_name):
    exp_dir = os.path.join(git_dir, "participants", par_name, exp_name)
    os.makedirs(exp_dir)

    return exp_dir

def create_cond_dir(exp_dir, par_name, exp_name):
    cond_dir = os.path.join(exp_dir, exp_name+"_conditions")
    os.makedirs(cond_dir)

    return cond_dir

def create_experiment_order(save_dir):
    exp_block1 = ["vSAT", "king_devick", "audio_narrative"]
    exp_block2 = ["n_back", "tower_of_london", "video_narrative_cmiyc"]
    exp_block3 = ["go_no_go", "resting_state", "video_narrative_sherlock"]
    exp_blocks = [exp_block1, exp_block2, exp_block3]

    # randomize order of first two experiments in each block
    for i, exp_block in enumerate(exp_blocks):
        exp_block_copy = exp_block[0:2]
        np.random.shuffle(exp_block_copy)    
        exp_block[0:2] = exp_block_copy  
        exp_blocks[i] = exp_block

    # randomize order of experiment block
    np.random.shuffle(exp_blocks) 

    par = os.path.basename(save_dir)
    filepath = os.path.join(save_dir, f"{par}_" + "experiment_order.txt")                        
    exp_file = open(filepath, 'w')

    for i, exp_block in enumerate(exp_blocks):
        exp_file.write(f"Block {i+1}\n" + "-------\n")
        exp_file.write(f"{exp_block[0]}\n" + f"{exp_block[1]}\n" + f"{exp_block[2]}\n" + "\n") 

    experiment_order = [exp for exp_block in exp_blocks for exp in exp_block]
   
    return experiment_order

def create_marker_order(save_dir, experiment_order):
    marker_dict_path = os.path.join(os.getcwd(), "marker_dict.csv")

    df = pd.read_csv(marker_dict_path)
    marker_strs = df["marker_str"]
    marker_vals = df["marker_val"]

    all_marker_info = zip(marker_strs, marker_vals)
    marker_order = {}
    marker_num = 1
    for i, exp in enumerate(experiment_order):
        for marker_info in zip(marker_strs, marker_vals):
            start_str = exp + "_start"
            end_str = exp + "_end"
            if start_str in marker_info[0]:
                marker_order[marker_info[1]] = marker_num
                marker_num += 1
            elif end_str in marker_info[0]:
                marker_order[marker_info[1]] = marker_num
                marker_num += 1

    par = os.path.basename(save_dir)
    filepath = os.path.join(save_dir, f"{par}_" + "marker_order.json")  
    
    with open(filepath, "w") as marker_order_json_file:
        json.dump(marker_order, marker_order_json_file)

def create_master_cond_csv(save_dir, cond_path_list):
    par = os.path.basename(save_dir)
    writer = pd.ExcelWriter(os.path.join(save_dir, f"{par}_conditions.xlsx"))
    
    for csv_filepath in cond_path_list:
        df = pd.read_csv(csv_filepath)
        sheet_name = Path(csv_filepath).stem
        if len(sheet_name) >= 31:
            print("ERROR")
        df.to_excel(writer, sheet_name=sheet_name)
    writer.save()  

def create_master_task_order_csv(save_dir, task_order_path_list):
    par = os.path.basename(save_dir)
    writer = pd.ExcelWriter(os.path.join(save_dir, f"{par}_task_orders.xlsx"))
    
    for csv_filepath in task_order_path_list:
        df = pd.read_csv(csv_filepath)
        sheet_name = Path(csv_filepath).stem
        if len(sheet_name) >= 31:
            print("ERROR")
        df.to_excel(writer, sheet_name=sheet_name)
    writer.save()

def gen_exp_dirs(num_pars):
    main_dir = os.getcwd()
    git_dir = os.path.dirname(main_dir)
    
    for par_num in range(num_pars):
        par_num += 1  # start participants at 1, not 0
        par = f"participant_{par_num:02d}"
        par_dir = os.path.join(git_dir, "participants", par)
        os.makedirs(par_dir)

        experiment_order = create_experiment_order(save_dir=par_dir)
        print("Completed ", par, " ", "experiment_order_file")
        create_marker_order(save_dir=par_dir, experiment_order=experiment_order)
        print("Completed ", par, " ", "marker_order_file")

        csv_filepath_list = []
        task_order_path_list = []

        for exp in experiment_order:
            if exp == "audio_narrative":
                create_exp_dir(git_dir, par, exp)
                print("Completed ", par, " ", exp)

            elif exp == "go_no_go":
                exp_dir = create_exp_dir(git_dir, par, exp)
                cond_dir = create_cond_dir(exp_dir, par, exp)

                go_stim_dir = os.path.join(git_dir, exp, "go_stimuli")
                no_go_stim_dir = os.path.join(git_dir, exp, "no_go_stimuli")

                num_rows = 20
                for i in range(2):
                    filepath = go_stim_csv(num_rows=num_rows, save_dir=cond_dir, go_stim_dir=go_stim_dir)
                    csv_filepath_list.append(filepath)
                    time.sleep(0.01)
                    filepath = GNG_stim_csv(num_rows=num_rows, save_dir=cond_dir, go_stim_dir=go_stim_dir, no_go_stim_dir=no_go_stim_dir)
                    csv_filepath_list.append(filepath)

                filepath = GNG_task_order_csv(cond_dir=cond_dir, save_dir=exp_dir)
                task_order_path_list.append(filepath)
                
                print("Completed ", par, " ", exp)

            elif exp == "king_devick":
                exp_dir = create_exp_dir(git_dir, par, exp)

                king_devick_resp(par=par, save_dir=exp_dir)
                print("Completed ", par, " ", exp)

            elif exp == "n_back":
                exp_dir = create_exp_dir(git_dir, par, exp)
                cond_dir = create_cond_dir(exp_dir, par, exp)

                conditions = [0, 1, 2]
                num_rows = 20

                for condition in conditions:
                    if condition == 0:
                        for i in range(3):
                            filepath = zero_back_csv(save_dir=cond_dir, num_rows=num_rows, match_prop=0.4)
                            csv_filepath_list.append(filepath)
                            time.sleep(0.01)
                    if condition == 1:
                        for i in range(3):
                            filepath = one_back_csv(save_dir=cond_dir, num_rows=num_rows, match_prop=0.4)
                            csv_filepath_list.append(filepath)
                            time.sleep(0.01)
                    if condition == 2:
                        for i in range(3):
                            filepath = two_back_csv(save_dir=cond_dir, num_rows=num_rows, match_prop=0.4)
                            csv_filepath_list.append(filepath)
                            time.sleep(0.01)

                filepath = n_back_task_order_csv(cond_dir=cond_dir, save_dir=exp_dir)
                task_order_path_list.append(filepath)
                
                print("Completed ", par, " ", exp)

            elif exp == "resting_state":
                exp_dir = create_exp_dir(git_dir, par, exp)

                filepath = resting_state_task_order_csv(save_dir=exp_dir)
                task_order_path_list.append(filepath)    

                print("Completed ", par, " ", exp)

            elif exp == "tower_of_london":
                exp_dir = create_exp_dir(git_dir, par, exp)
                cond_dir = create_cond_dir(exp_dir, par, exp)

                zero_move_dir = os.path.join(git_dir, exp, "zero_move_stimuli")
                two_move_dir = os.path.join(git_dir, exp, "multi_move_stimuli", "stim_two_move")
                not_two_move_dir = os.path.join(git_dir, exp, "multi_move_stimuli", "stim_not_two_move")
                
                num_rows = 6

                for i in range(3):
                    filepath = ZM_stim_csv(num_rows=num_rows, save_dir=cond_dir, zero_move_dir=zero_move_dir)
                    csv_filepath_list.append(filepath)
                    time.sleep(0.01)
                    filepath = MM_stim_csv(num_rows=num_rows, save_dir=cond_dir, two_move_dir=two_move_dir, not_two_move_dir=not_two_move_dir) 
                    csv_filepath_list.append(filepath)

                filepath = TOL_task_order_csv(cond_dir=cond_dir, save_dir=exp_dir)
                task_order_path_list.append(filepath)

                print("Completed ", par, " ", exp)

            elif exp == "video_narrative_cmiyc":
                exp_dir = create_exp_dir(git_dir, par, exp)
                print("Completed ", par, " ", exp)

            elif exp == "video_narrative_sherlock":
                exp_dir = create_exp_dir(git_dir, par, exp)
                print("Completed ", par, " ", exp)

            elif exp == "vSAT":
                exp_dir = create_exp_dir(git_dir, par, exp)
                cond_dir = create_cond_dir(exp_dir, par, exp)

                num_rows = 30
                SAT = True

                for i in range(2):
                    filepath = vSAT_stim_csv(num_rows=num_rows, save_dir=cond_dir, SAT=True)
                    csv_filepath_list.append(filepath)
                    time.sleep(0.01)

                for i in range(2):
                    filepath = vSAT_stim_csv(num_rows=num_rows, save_dir=cond_dir, SAT=False)
                    csv_filepath_list.append(filepath)
                    time.sleep(0.01)

                filepath = vSAT_task_order_csv(cond_dir=cond_dir, save_dir=exp_dir)
                task_order_path_list.append(filepath)

                print("Completed ", par, " ", exp)

        create_master_cond_csv(save_dir=par_dir, cond_path_list=csv_filepath_list)
        print("Completed ", par, " ", "master_conditions_file")
        create_master_task_order_csv(save_dir=par_dir, task_order_path_list=task_order_path_list)
        print("Completed ", par, " ", "master_task_order_file\n")


In [8]:
cwd = os.getcwd()
par_dir = os.path.join(os.path.dirname(cwd), "participants")
num_pars = 50

try:
    shutil.rmtree(par_dir)
except:
    print("Dir does not exist. Creating new dir...")

gen_exp_dirs(num_pars=num_pars)

os.chdir(cwd)

Completed  participant_01   experiment_order_file
Completed  participant_01   marker_order_file
Completed  participant_01   n_back
Completed  participant_01   tower_of_london
Completed  participant_01   video_narrative_cmiyc
Completed  participant_01   resting_state
Completed  participant_01   go_no_go
Completed  participant_01   video_narrative_sherlock
Completed  participant_01   vSAT
Completed  participant_01   king_devick
Completed  participant_01   audio_narrative
Completed  participant_01   master_conditions_file
Completed  participant_01   master_task_order_file

Completed  participant_02   experiment_order_file
Completed  participant_02   marker_order_file
Completed  participant_02   tower_of_london
Completed  participant_02   n_back
Completed  participant_02   video_narrative_cmiyc
Completed  participant_02   vSAT
Completed  participant_02   king_devick
Completed  participant_02   audio_narrative
Completed  participant_02   go_no_go
Completed  participant_02   resting_state
Co